In [2]:
#Part 2 
#(1) Write a function to evaluate the emission parameters based on data set X and Y

#Input: Directory of files
#Output: X data set and Y data set

def get_XY(datafile_dir):
    f=open(datafile_dir)
    f_content=f.read()
    X = []
    Y = []
    xi = []
    yi = []
    
    for data_pair in f_content.split('\n'):
        
        if data_pair=='':
            X.append(xi)
            Y.append(yi)
            xi = []
            yi = []
            
        else:
            xij,yij = data_pair.split(" ")
            xi.append(xij)
            yi.append(yij)
            
    return (X,Y)

#Input: Dataset X and Y
#Output: Emission parameters based on Dataset X and Y
def train_emission_param(X,Y):
    o_unique = []
    T = ['O', 'B-positive', 'I-positive','B-negative','I-negative','B-neutral','I-neutral']
    
    for xi in X:
        
        for o in xi:
            if o not in o_unique:
                o_unique.append(o)
        
    #print (o_unique)
    count_y_dic = {'O':0, 'B-positive':0, 'I-positive':0,'B-negative':0,'I-negative':0,'B-neutral':0,'I-neutral':0}
    count_x_y_dic = {}
    em_dic = {}
    for i in range(len(X)):
        xi = X[i]
        yi = Y[i]
        
        for j in range(len(xi)):
            key = (xi[j],yi[j])
            key_deno = yi[j]
            origin = count_y_dic[key_deno] 
            count_y_dic[key_deno] = origin + 1
            
            if key not in count_x_y_dic:
                count_x_y_dic[key] = 1
            else:
                value = count_x_y_dic[key]
                count_x_y_dic[key] = value + 1
   
    for o in o_unique:
        
        for state in T:
            key = (o,state)
            if key not in count_x_y_dic:
                em_dic[key] = 1/float(count_y_dic[state]+1)#Fix the problem where the word is not in the training set
            else:
                em_dic[key] =float(count_x_y_dic[key])/float(count_y_dic[state])
            print (key, em_dic[key])
    print ('hi')
    
    return em_dic
                
(X,Y)=get_XY('EN/train')
train_emission_param(X,Y)


    

('Ray', 'O') 5.723770820216359e-05
('Ray', 'B-positive') 0.0026455026455026454
('Ray', 'I-positive') 0.004273504273504274
('Ray', 'B-negative') 0.008333333333333333
('Ray', 'I-negative') 0.0196078431372549
('Ray', 'B-neutral') 0.0017211703958691911
('Ray', 'I-neutral') 0.0013966480446927375
('J', 'O') 5.723770820216359e-05
('J', 'B-positive') 0.0026455026455026454
('J', 'I-positive') 0.004291845493562232
('J', 'B-negative') 0.008403361344537815
('J', 'I-negative') 0.0196078431372549
('J', 'B-neutral') 0.0008605851979345956
('J', 'I-neutral') 0.004189944134078212
('Breaks', 'O') 5.7240984544934173e-05
('Breaks', 'B-positive') 0.0026455026455026454
('Breaks', 'I-positive') 0.004273504273504274
('Breaks', 'B-negative') 0.008333333333333333
('Breaks', 'I-negative') 0.0196078431372549
('Breaks', 'B-neutral') 0.0008598452278589854
('Breaks', 'I-neutral') 0.001394700139470014
('His', 'O') 0.00011448196908986835
('His', 'B-positive') 0.0026455026455026454
('His', 'I-positive') 0.00427350427350

{('Timelapse', 'B-positive'): 0.0026455026455026454,
 ('themselves', 'B-negative'): 0.008333333333333333,
 ('libyans', 'I-neutral'): 0.001394700139470014,
 ('manufacturing', 'I-positive'): 0.004273504273504274,
 ('Listener', 'I-neutral'): 0.001394700139470014,
 ('Phoenix', 'I-positive'): 0.004273504273504274,
 ('google', 'O'): 5.723770820216359e-05,
 ("I'd", 'B-positive'): 0.0026455026455026454,
 ('love', 'B-positive'): 0.0026455026455026454,
 ('Daily', 'B-neutral'): 0.0008598452278589854,
 ('bunch', 'B-negative'): 0.008333333333333333,
 ('since', 'I-positive'): 0.004273504273504274,
 ('http://t.co/6DOMivEH', 'B-negative'): 0.008333333333333333,
 ('Theater', 'B-negative'): 0.008333333333333333,
 ('QLD', 'B-positive'): 0.0026455026455026454,
 ('http://t.co/4xjXtOWP', 'B-positive'): 0.0026455026455026454,
 ('http://t.co/Ck5zYeME', 'I-neutral'): 0.001394700139470014,
 ('Irvine', 'O'): 5.723770820216359e-05,
 ('Healthymagination', 'I-neutral'): 0.001394700139470014,
 ('tool', 'B-negative')